### js逆向笔记

#### 加密算法：

In [ ]:
    1.线性散列算法 MD5加密
    2.对称线性加密 AES,DES
    3.非对称线性加密 RSA
    4.base64伪加密

#### 特征:

    1.MD5  加密后产生的是一个16位或者32位的数据，js包中算法流程常常出现大量类似于：
    a = md5_gg(a,d,c,b,x[ i+1],5,-165796510)，并且一般用于注册账号时设置密码的加密，或者登陆账号时对密码的加密。破解成本极高，安全性很强。
    2.AES比DES更安全，但是企业级使用DES就够。现在广泛使用AES，对称加密方式就是加密和解密使用同一组密钥, DES加密后长度位数是8的整数倍，AES加密后是16的整数倍。  两种加密方法只需要用CryptoJS.AES<=>CryptoJS.DES.破解起来只要算力够强大，是可以破解的。若56位，则可能性有2的56次方个。js包中若包含DES/AES.encrypt,DES/AES.decrypt，则是DES加密或者AES加密
    3.加密方式为公钥加密，私钥解密，公钥可以公开，私钥自己保留，算法强度复杂，安全性依赖于算法和密钥，因为算法复杂，所以加密解密速度不如对称加密，一般支付密码使用RSA加密。公钥私钥可以在线生成：http://web.chacuo.net/netrsakeypair js包中算法流程会包含setpublickey和setprivatekey
    4.base64伪加密是一种编码方式，是使用64个字符来表示任意二进制数据的方法 加密base64.encode(),解密base64.decode(),多用于图片链接加密，详情页链接加密等。
    

#### js逆向步骤

In [ ]:
一.工具与环境
    1.配置好nodejs环境
    2.下载js调试工具，此处使用发条js调试工具
    3.安装pyexecjs模块

In [ ]:
二.js逆向具体流程
    1.发送网络请求，遇到了加密过的参数
    2.搜索该加密参数名，而非值，不要直接ctrl+F搜索，这样无法打断点，使用谷歌浏览器调试界面右上角的‘关闭x’按钮左边的菜单功能，即三个点的
    那个标志。使用search。挨个找包含此参数名的js包。如果很多，可以尝试加上空格=或者：一起搜索，点进去后点击该页面左下角的大括号标志，将页面代
    码格式化。
    3.在该小界面点击一下，ctrl+f,再次搜索加密参数名。
    4.在涉及加密参数的语句附近寻找加密算法关键字，若有，则打断点(单击行号)，不清楚的也打断点。
      JS混淆：将js核心的相关代码进行变相的加密（表现为既不符合某种语法要求，也不是一个字符串，可能是密文，也可能是特殊格式的一组值）
      JS反混淆：a.反混淆线上工具（多余的空格标点符号之类，造成效果不理想）
                b.浏览器自带的反混淆工具设置（推荐），即开发者工具中选择sources,点settings,点sources,勾选上search in anonymous and content scripts
                  关闭该页面，关闭js文件页面，重新搜索关键字，VM开头的文件即为反混淆后的结果
      公钥密钥之类要么在前台页面，要么是通过某些请求请求到的
    5.再次发送网络请求，看运行到哪个断点停下来
    6.在停下来的断点处分析，若有什么函数，可点击进入该函数继续分析，直到分析到某函数传入我们的明文数据，return的是某些参数，那么就是这个函
    数进行的参数加密。
    7.将该函数内容粘贴到【发条js调试工具】中，运行，会发现错误。因为该函数中可能还涉及其他参数，所以我们将包含此函数以及其他函数的上一级函
    数这个大一级的大括号{}中的函数内容全部粘贴过去。再运行。
    8.运行后若存在未定义的变量，将其定义为空字典即可，如：  var n ={},若存在js内置变量未定义的话，定义为this，如navigator = this;
      若涉及到加密参数的函数是闭包语法：形式为 (function(x)) {一大堆}(一大堆)； 则直接复制整个闭包函数即可。常识：.serializeArray()是js语法中做序列化处理的
    9.若未成功，从第5点继续尝试。
    10.略微改写js代码，使其可以被调用，将return加密结果的那一行改为 例如，将n.celler=function(a+b-c)改为function getpwd(a+b-c),调用时
    直接getpwd('123456')就可以将123456加密为密文。

In [ ]:
三.python环境中加载js逆向代码
    1.以上为在发条js调试工具中的操作，如何将该函数运行到python环境中呢？
    2.新建一个js空文件，将上述函数粘贴进去。
    3.爬虫文件中：
    import execjs
    node=execjs.get()  #实例化一个node对象
    ctx=node.compile(open('上述js文件路径及js文件名'，encoding='utf-8').read())  #js源文件编译
    funcname='getpwd({})'.format('123456') #向函数传递参数
    pwd=ctx.eval(funcname)  #执行js函数
    print(pwd)
    